In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 500

### Загрузим датасет с машинами. Цель - верно восстанавливать для каждой из них цену продажи!

In [2]:
data = pd.read_csv('autos.csv')

data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [3]:
### Колонка с тергетом - "selling price"

X = data.drop("selling_price", axis=1)
y = data["selling_price"]

### Будем замерять MSLE!
### Поэтому прологарифмируем таргет
### А после оптимизируем MSE

y = y.apply(np.log1p)

In [4]:
### Разделим выборку на трейн и тест!

from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

__Задание__ 

Реализуйте свой MeanTargetEncoder с добавленем некоторого шума!

Однажды в лекционном материале, обсуждая счетчики, мы говорили с вами о том, что из-за них модели могут переобучаться. Один из способов бороться с этим - валидировать расчеты среднего таргета (стратегия отложенной выборки / расчеты на кросс-валидации). Но есть еще проще!

Можно просто к значению счетчика добавить случайный шум (зашумить данные)!

Напомним, что рассчитываться новые признаки должны по такой формуле:

$$
g_j = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}{l} + C * \epsilon
$$



Пусть шум будет случайной величиной из нормального стандартного распределения, то есть $\epsilon \sim N(0, 1) $, а $ C = 0.006$.

Создавая свой класс-трансформер, наследуйтесь от классов `BaseEstimator, TransformerMixin` из `sklearn.base`. Трансформер не должен модифицировать передаваемую ему выборку inplace, а все необходимые статистики нужно считать только по обучающей выборке в методе `fit`. Ваш трансформер должен принимать при инициализации список из категориальных признаков и список из числовых признаков. 

Если для какого-то признака в тестовой выборке отсутствует значение, трансформер должен поставить там 0.

На выходе должен получиться датасет того же размера с измененными категориальными признаками.

Класс MeanTargetEncoderNoise должен иметь следующую сигнатуру:



In [26]:
# from sklearn.base import BaseEstimator, TransformerMixin
# 
# class MeanTargetEncoderNoise(BaseEstimator, TransformerMixin):
# 
#     def __init__(self, categorical, numeric):              
#         self.categorical = categorical
#         self.numeric_columns = numeric
# 
#     def fit(self, X, y):
#         X_fit = X.copy()
#         y_fit = y.copy()
# 
# 
#         X_with_target = pd.concat((X_fit, y_fit), axis=1) 
# 
#         ### Сгенерим колонки к которым применим Mean-Target-Encoding
#         self.cols_for_mte = [col for col in self.categorical]
# 
#         self.dict_of_means = {col : X_with_target.groupby(col)['selling_price'].mean() + 0.006 * np.random.normal(0,1)
#                               for col in self.cols_for_mte}
# 
#         return self
# 
#     def transform(self, df):
#         ### Your code is here
#         X_ = X.copy()
# 
#         for col in self.cols_for_mte:
#                 X_[col] = X_[col].map(self.dict_of_means[col])
# 
#                 mean_value = self.dict_of_means[col].values.mean()
# 
#                 X_[col] = X_[col].fillna(mean_value)
#             
#         all_ohe = list(itertools.chain(*list(self.dict_of_means.values())))
# 
# 
#         missing_columns = [x 
#                            for x in all_ohe
#                            if x not in X_.columns
#                            and
#                            x not in self.numeric_columns]
# 
# 
#         if len(missing_columns) != 0:
# 
#             zeros = np.zeros((X_.shape[0], len(missing_columns)))
#             zeros = pd.DataFrame(zeros,
#                                  columns=missing_columns,
#                                  index=X_.index)
# 
#             X_ = pd.concat((X_, zeros), axis=1)
# 
#         return X_[sorted(X_.columns)]

In [28]:
# from sklearn.base import BaseEstimator, TransformerMixin
# import itertools
# 
# class CustomFunctionTransformer(BaseEstimator, TransformerMixin):
#     
#     def __init__(self, categorical, numeric):              
#         self.object_columns = categorical
#         self.numeric_columns = numeric
#         
#                 
#     def fit(self,
#             X,
#             y):
#         
#         X_fit = X.copy()
#         y_fit = y.copy()
# 
#         
#         X_with_target = pd.concat((X_fit, y_fit), axis=1)
#         
#         ### Сгенерим колонки к которым применим One-Hot-Encoding
#         self.cols_for_ohe = [col for col in self.object_columns
#                              if 
#                              X_with_target[col].nunique() <= 10]
#         
#         ### Запомним все ohe колонки и их названия!
#         self.ohe_names = {col : sorted([f"{col}_{value}" for value in X_with_target[col].unique()])
#                           for col in self.cols_for_ohe}
#         
#         
#         ### Сгенерим колонки к которым применим Mean-Target-Encoding
#         self.cols_for_mte = [col for col in self.object_columns
#                              if X_with_target[col].nunique() > 10]
#         
#         ### Посчитаем на валидации средние значения таргета
#         self.dict_of_means = {col : X_with_target.groupby(col)['selling_price'].mean() + np.random.normal(0,1)
#                               for col in self.cols_for_mte}
#         
#         
#         return self
#     
#     def transform(self,
#                   X,
#                   y=None):
#         
#         X_ = X.copy()
#         
#         
#         data_part = pd.get_dummies(X_[self.cols_for_ohe],
#                                    prefix=self.cols_for_ohe)
#         
#         data_part_cols = data_part.columns
#         
#         X_ = X_.drop(self.cols_for_ohe, axis=1)
#         X_ = pd.concat((X_, data_part), axis=1)
#         
#     
#         for col in self.cols_for_mte:
#                 X_[col] = X_[col].map(self.dict_of_means[col])
#                 
#                 mean_value = self.dict_of_means[col].values.mean()
#                 
#                 X_[col] = X_[col].fillna(mean_value)
#                 
#             
#             
#         all_ohe = list(itertools.chain(*list(self.ohe_names.values())))
#         
#         missing_columns = [x 
#                            for x in all_ohe
#                            if x not in X_.columns
#                            and
#                            x not in self.numeric_columns]
# 
#         extra_columns = [x
#                          for x in data_part_cols
#                          if x not in all_ohe]
#         
#         ### Новые категории необходимо убрать
#         X_ = X_.drop(extra_columns, axis=1)
#     
#         ### Отсутствующие категории (бинарные колонки)
#         ### необходимо добавить: заполним их просто нулями
#         
#         if len(missing_columns) != 0:
# 
#             zeros = np.zeros((X_.shape[0], len(missing_columns)))
#             zeros = pd.DataFrame(zeros,
#                                  columns=missing_columns,
#                                  index=X_.index)
# 
#             X_ = pd.concat((X_, zeros), axis=1)
#             
#         return X_.loc[:, sorted(map(str, X_.columns))]


Разделите колонки на вещественные и категориальные. Приведите все категориальные колонки к типу `object`.

Далее применим наш кодировщик к `X_train, X_test`, так же как например мы применяем `StandardScaler`, чтобы проверить работоспособность нашего класса. Установите зерно датчика случайный чисел `np.random.seed(1)`.

После того, как вы изменили обучающую и тестовую выборки, сохраните первые 10 строк полученного промежуточного датафрейма обучающей выборки (`X_train`) в файл в формате csv с сепаратором `;`. Не забудьте индекс. Отправьте полученный файл в форму ниже.

Список колонок которые должны быть в файле для сдачи:
```py
cols = [
    "km_driven",
    "name",
    "year",
    "fuel",
    "seller_type",
    "transmission",
    "owner"
]
```

### Ваше решение


Разделение колонок на категориальные и числовые.

In [30]:
object_cols = ['name', 'year', 'fuel', 'seller_type', 'transmission', 'owner']
num_cols = ['km_driven']

X.head()

,name,year,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,141000,Diesel,Individual,Manual,Second Owner


Реализация класса MeanTargetEncoderNoise.

In [31]:
from sklearn.base import BaseEstimator, TransformerMixin
class MeanTargetEncoderNoise(BaseEstimator, TransformerMixin):
    
    def __init__(self, categorical, numeric):              
        self.categorical = categorical
        self.numeric = numeric
        self.mapping = {}
    
    def fit(self, X, y):
        np.random.seed(1)
        
        for col in self.categorical:
            mean_target = y.groupby(X[col]).mean()
            noise = 0.006 * np.random.normal(size=len(mean_target))
            self.mapping[col] = mean_target + noise
        
        return self
        
    def transform(self, df):
        temp = df.copy()
        
        for col in self.categorical:
            temp[col] = temp[col].map(self.mapping[col]).fillna(0)
            
        return temp

Проверка работы трансформера.

In [32]:
np.random.seed(1)
transformer = MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)

transformer.fit(X_train, y_train)

train = transformer.transform(X_train)
test = transformer.transform(X_test)

train.head(10)

,name,year,km_driven,fuel,seller_type,transmission,owner
3294,13.483692,13.436559,50000,13.093756,12.615863,13.771135,12.977730
2290,12.117029,11.903115,70000,12.453832,12.615863,13.771135,12.977730
874,12.310456,13.328864,50000,12.453832,12.615863,12.639805,12.977730
1907,12.491443,13.042359,92198,12.453832,13.152824,12.639805,12.463313
3244,12.390906,12.870886,3240,12.453832,12.615863,12.639805,12.463313
1089,12.687432,13.436559,10000,12.453832,13.152824,12.639805,12.977730
3902,11.698702,11.503504,90000,12.453832,12.615863,12.639805,11.878390
2215,11.120678,11.503504,79000,12.453832,12.615863,12.639805,12.463313
3862,13.172879,13.328864,99700,13.093756,12.615863,12.639805,12.977730
705,13.004194,12.241213,124000,13.093756,12.615863,12.639805,12.463313


In [33]:
train.head(10).to_csv('c.csv', sep=';', index=False)

Обучите несколько деревьев, перебирая максимальную глубину алгоритма из списка `max_depth_list`, а остальные параметры оставьте дефолтными. Выведите лучшее значение гиперпараметра. Постройте график зависимости MSLE на тестовой выборке от значения гиперпараметра. Воспользуйтесь `Pipeline` без `GridSearch`. Проделайте то же самое с `min_samples_split`, `min_impurity_decrease`, `max_leaf_nodes`. (по 2б на каждый параметр)

In [34]:
max_depth_list = [3, 5, 8, 12]
min_samples_split_list = [10, 50, 100, 500]
min_impurity_decrease_list = [0, 0.1, 0.15, 0.2]
max_leaf_nodes_list = [100, 200, 500]

In [42]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline

np.random.seed(1)
list_d = list()
### Your code is here
for max_depth in max_depth_list:

    pipe = Pipeline([("custom_transformer",
                  MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
                 ("decision_tree", 
                  DecisionTreeRegressor(max_depth=max_depth))])

    pipe.fit(X_train, y_train)
    
    preds = pipe.predict(X_test)
    test_error = np.mean((preds - y_test)**2)

    list_d.append(test_error.round(2))
list_d


C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is

[0.8, 1.44, 1.99, 1.99]

In [41]:
list_d = list()
### Your code is here
for min_sample in min_samples_split_list:

    pipe = Pipeline([("custom_transformer",
                  MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
                 ("decision_tree", 
                  DecisionTreeRegressor(min_samples_split=min_sample))])

    pipe.fit(X_train, y_train)
    
    preds = pipe.predict(X_test)
    test_error = np.mean((preds - y_test)**2)

    list_d.append(test_error.round(2))
list_d

C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is

[1.43, 1.44, 0.95, 0.81]

In [40]:
list_d = list()
### Your code is here
for min_imp in min_impurity_decrease_list:

    pipe = Pipeline([("custom_transformer",
                  MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
                 ("decision_tree", 
                  DecisionTreeRegressor(min_impurity_decrease=min_imp))])

    pipe.fit(X_train, y_train)
    
    preds = pipe.predict(X_test)
    test_error = np.mean((preds - y_test)**2)

    list_d.append(test_error.round(2))
list_d

C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is

[1.98, 0.52, 0.52, 0.52]

In [43]:
list_d = list()
### Your code is here
for max_leaf in max_leaf_nodes_list:

    pipe = Pipeline([("custom_transformer",
                  MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
                 ("decision_tree", 
                  DecisionTreeRegressor(max_leaf_nodes=max_leaf))])

    pipe.fit(X_train, y_train)
    
    preds = pipe.predict(X_test)
    test_error = np.mean((preds - y_test)**2)

    list_d.append(test_error.round(2))
list_d

C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is

[1.99, 1.98, 1.98]

Подберите лучшую комбинацию параметров, используя `GridSearchCV` и набор массивов значений параметров из предыдущего задания. Для лучшей комбинации посчитайте MSLE на тестовой выборке. Получились ли лучшие параметры такими же, как если бы вы подбирали их по-отдельности при остальных гиперпараметрах по умолчанию (предыдущее задание)? (2б)

In [49]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "decision_tree__max_depth": [3, 5, 8, 12],
    "decision_tree__min_samples_split": [10, 50, 100, 500],
    "decision_tree__min_impurity_decrease": [0, 0.1, 0.15, 0.2],
    "decision_tree__max_leaf_nodes": [100, 200, 500]
}
np.random.seed(1)

### Your code is here
pipe = Pipeline([("custom_transformer",
                  MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
                 ("decision_tree", 
                  DecisionTreeRegressor())])
search = GridSearchCV(pipe, 
                      param_grid, 
                      cv=3,
                      scoring='neg_mean_squared_error')

search.fit(X_train, y_train)


C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('custom_transformer',
                                        MeanTargetEncoderNoise(categorical=['name',
                                                                            'year',
                                                                            'fuel',
                                                                            'seller_type',
                                                                            'transmission',
                                                                            'owner'],
                                                               numeric=['km_driven'])),
                                       ('decision_tree',
                                        DecisionTreeRegressor())]),
             param_grid={'decision_tree__max_depth': [3, 5, 8, 12],
                         'decision_tree__max_leaf_nodes': [100, 200, 500],
                         'decision_tree__min_impurity_decrease': [0, 0.1, 0.15,
                                                                  0.2],
                         'decision_tree__min_samples_split': [10, 50, 100,
                                                              500]},
             scoring='neg_mean_squared_error')

In [51]:
print(f"Best parameter (CV score={search.best_score_:.2f}):")
print(search.best_params_)

Best parameter (CV score=-0.59):
{'decision_tree__max_depth': 3, 'decision_tree__max_leaf_nodes': 100, 'decision_tree__min_impurity_decrease': 0.1, 'decision_tree__min_samples_split': 10}


In [52]:
best_model = search.best_estimator_

# Предсказание на тестовой выборке
y_pred = best_model.predict(X_test)
y_pred

C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


array([13.38252765, 13.38252765, 13.38252765, 12.08653247, 13.38252765,
       12.08653247, 12.08653247, 12.08653247, 12.08653247, 12.08653247,
       13.38252765, 12.08653247, 12.08653247, 12.08653247, 12.08653247,
       12.08653247, 12.08653247, 12.08653247, 12.08653247, 13.38252765,
       12.08653247, 13.38252765, 12.08653247, 12.08653247, 12.08653247,
       12.08653247, 13.38252765, 13.38252765, 13.38252765, 12.08653247,
       13.38252765, 12.08653247, 13.38252765, 13.38252765, 13.38252765,
       12.08653247, 12.08653247, 13.38252765, 12.08653247, 12.08653247,
       12.08653247, 12.08653247, 12.08653247, 12.08653247, 12.08653247,
       13.38252765, 13.38252765, 12.08653247, 13.38252765, 12.08653247,
       12.08653247, 13.38252765, 12.08653247, 12.08653247, 13.38252765,
       13.38252765, 12.08653247, 13.38252765, 13.38252765, 12.08653247,
       13.38252765, 12.08653247, 12.08653247, 12.08653247, 12.08653247,
       13.38252765, 12.08653247, 12.08653247, 12.08653247, 12.08

In [53]:
print(f"Качество лучшей модели на финальном тесте: {search.score(X_test, y_test)}")

Качество лучшей модели на финальном тесте: -0.5229485887238029


C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\Vica\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is